In [1]:
import pandas as pd
import numpy as np

from github import Github
from settings import *

In [2]:
# Function to replace continent name with continent id
def getContinentId(x):
    if str(x) in continent_dict.keys():
        return continent_dict[str(x)]

# Function to fix vaccine names
def fix_vaccines(x):
    if "Sinopharm/Wuhan" in str(x) or "Sinopharm/Beijing" in str(x):
        return "Sinopharm"
    elif "Pfizer/ BioNTech" in str(x):
        return "Pfizer/BioNTech"
    else:
        return str(x)

# Function to replace vaccine name with vaccine id
def getVaccineId(x):
    if str(x) in vaccine_dict.keys():
        return vaccine_dict[str(x)]

# Function to replace country name with country id
def getCountryId(x):
    if str(x) in country_dict.keys():
        return country_dict[str(x)]

In [3]:
### Creating continent.csv ###
country_df = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/jhu/locations.csv")

country_df = country_df.dropna()
country_df = country_df.reset_index(drop=True)

continent_df = country_df.drop(columns=["location", "Country/Region", "population_year", "population"])
continent_df = continent_df.drop_duplicates(ignore_index = True)

# We shift index by 1 in order to input csv into MySQL
continent_df.index = continent_df.index + 1

continent_df.to_csv("continent.csv")
print("Created 'continent.csv'")

Created 'continent.csv'


In [4]:
### Creating country.csv ###
country_df = country_df.drop(columns=["Country/Region", "population_year"])
# Fix population data type
country_df = country_df.astype({'population': 'int'})
# replace continent with continent_id
continent_dict = dict()
for i in range(len(continent_df)):
    continent_name = continent_df.loc[i + 1, "continent"]
    continent_dict[continent_name] = i + 1

country_df.continent = country_df.continent.apply(getContinentId)

country_df.index = country_df.index + 1

country_df.to_csv("country.csv")
print("Created 'country.csv'")

Created 'country.csv'


In [5]:
### Merging vaccine files from Github to create other csv files ###
token = getKey()
g = Github(token)

repo = g.get_repo("owid/covid-19-data")
file_list = repo.get_contents("public/data/vaccinations/country_data")
github_path = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/country_data/"
for i in range(len(file_list)):
    csv_file = str(file_list[i]).split("/")[-1].split(".")[0]+ ".csv"
    csv_file = csv_file.replace(" ", "%20")
    file_list[i] = github_path + csv_file

In [6]:
# Merging files into a dataframe called "world_df"
dataframe_list = []

for file in file_list:
    df = pd.read_csv(file)
    dataframe_list.append(df)

world_df = pd.concat(dataframe_list, ignore_index=True)

world_df = world_df.drop(columns=["source_url"])

In [7]:
world_df

,location,date,vaccine,total_vaccinations,people_vaccinated,people_fully_vaccinated
0,Afghanistan,2021-02-22,Oxford/AstraZeneca,0.0,0.0,NaN
1,Afghanistan,2021-02-28,Oxford/AstraZeneca,8200.0,8200.0,NaN
2,Afghanistan,2021-03-16,Oxford/AstraZeneca,54000.0,54000.0,NaN
3,Afghanistan,2021-04-07,Oxford/AstraZeneca,120000.0,120000.0,NaN
4,Afghanistan,2021-04-22,Oxford/AstraZeneca,240000.0,240000.0,NaN
...,...,...,...,...,...,...
9794,Zimbabwe,2021-05-05,Sinopharm/Beijing,576233.0,461023.0,115210.0
9795,Zimbabwe,2021-05-06,Sinopharm/Beijing,607355.0,478174.0,129181.0
9796,Zimbabwe,2021-05-07,Sinopharm/Beijing,640762.0,500422.0,140340.0
9797,Zimbabwe,2021-05-08,Sinopharm/Beijing,657838.0,509274.0,148564.0


In [8]:
# Fixing vaccine column
explode_df = world_df.assign(vaccine=world_df.vaccine.str.split(", ")).explode("vaccine", ignore_index=True)
explode_df.vaccine = explode_df.vaccine.apply(fix_vaccines)

In [9]:
### Creating vaccine.csv ###
vaccine_df = explode_df.drop(columns=["location", "date", "total_vaccinations", "people_vaccinated", "people_fully_vaccinated"])
vaccine_df.vaccine.unique().tolist()
vaccine_df = vaccine_df.drop_duplicates(ignore_index = True)

In [10]:
vaccine_df

,vaccine
0,Oxford/AstraZeneca
1,Pfizer/BioNTech
2,Sputnik V
3,Sinovac
4,Sinopharm
5,Moderna
6,Johnson&Johnson
7,Covaxin
8,CanSino
9,EpiVacCorona


In [11]:
# Adding dose
doses = [2, 2, 2, 2, 2, 2, 1, 2, 1, 2]
vaccine_df["doses"] = doses

In [12]:
vaccine_df

,vaccine,doses
0,Oxford/AstraZeneca,2
1,Pfizer/BioNTech,2
2,Sputnik V,2
3,Sinovac,2
4,Sinopharm,2
5,Moderna,2
6,Johnson&Johnson,1
7,Covaxin,2
8,CanSino,1
9,EpiVacCorona,2


In [13]:
vaccine_df.index = vaccine_df.index + 1

vaccine_df.to_csv("vaccine.csv")
print("Created 'vaccines.csv'")

Created 'vaccines.csv'


In [14]:
### Creating vaccine_brand_in_country.csv ###
brand_df = explode_df.drop(columns=["date", "total_vaccinations", "people_vaccinated", "people_fully_vaccinated"])
brand_df = brand_df.drop_duplicates(ignore_index=True)

In [15]:
# Creating vaccine dictionary to replace vaccine name with its id
vaccine_dict = dict()
for i in range(len(vaccine_df)):
    vaccine_name = vaccine_df.loc[i + 1, "vaccine"]
    vaccine_dict[vaccine_name] = i + 1

In [16]:
# Replace vaccine name with vaccine_id
brand_df.vaccine = brand_df.vaccine.apply(getVaccineId)

In [17]:
# Creating vaccine dictionary to replace country name with its id
country_dict = dict()
for i in range(len(country_df)):
    country_name = country_df.loc[i + 1, "location"]
    country_dict[country_name] = i + 1

In [18]:
# Replacing country name with country_id
brand_df.location = brand_df.location.apply(getCountryId)

brand_df = brand_df.dropna()
brand_df = brand_df.reset_index(drop=True)
brand_df = brand_df.astype({"location": "int"})

In [19]:
brand_df

,location,vaccine
0,1,1
1,2,2
2,2,1
3,2,3
4,2,4
...,...,...
350,183,1
351,186,3
352,187,1
353,189,1


In [20]:
brand_df.index = brand_df.index + 1

brand_df.to_csv("vaccine_brand_in_country.csv")
print("Created 'vaccine_brand_in_country.csv'")

Created 'vaccine_brand_in_country.csv'


In [21]:
### Creating total_vaccination_by_brand.csv ###
vax_brand_df = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations-by-manufacturer.csv")

In [22]:
# Getting id's
vax_brand_df.location = vax_brand_df.location.apply(getCountryId)
vax_brand_df.vaccine = vax_brand_df.vaccine.apply(getVaccineId)

In [23]:
vax_brand_df

,location,date,vaccine,total_vaccinations
0,35,2020-12-24,2,420
1,35,2020-12-25,2,5198
2,35,2020-12-26,2,8338
3,35,2020-12-27,2,8649
4,35,2020-12-28,2,8649
...,...,...,...,...
3637,181,2021-05-08,6,111094349
3638,181,2021-05-08,2,137216849
3639,181,2021-05-09,7,8933802
3640,181,2021-05-09,6,111933191


In [24]:
vax_brand_df.index = vax_brand_df.index + 1

vax_brand_df.to_csv("total_vaccination_by_brand.csv")
print("Created 'total_vaccination_by_brand.csv'")

Created 'total_vaccination_by_brand.csv'


In [25]:
### Creating daily_vaccination.csv ###
daily_df = world_df.drop(columns=["vaccine", "total_vaccinations", "people_fully_vaccinated"])
daily_df.location = daily_df.location.apply(getCountryId)
daily_df = daily_df.dropna()
daily_df = daily_df.reset_index(drop=True)
daily_df = daily_df.astype({'people_vaccinated': 'int'})
daily_df = daily_df.astype({'location': 'int'})

In [26]:
daily_df

,location,date,people_vaccinated
0,1,2021-02-22,0
1,1,2021-02-28,8200
2,1,2021-03-16,54000
3,1,2021-04-07,120000
4,1,2021-04-22,240000
...,...,...,...
8010,190,2021-05-05,461023
8011,190,2021-05-06,478174
8012,190,2021-05-07,500422
8013,190,2021-05-08,509274


In [27]:
daily_df.index = daily_df.index + 1

daily_df.to_csv("daily_vaccination.csv")
print("Created 'daily_vaccination.csv'")

Created 'daily_vaccination.csv'


In [28]:
### Creating covid_data.csv ###
covid_df = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/jhu/full_data.csv")
covid_df = covid_df.drop(columns=["weekly_cases", "weekly_deaths", "biweekly_cases", "biweekly_deaths"])
covid_df = covid_df.astype({'new_cases': 'Int64'})
covid_df = covid_df.astype({'new_deaths': 'Int64'})
covid_df = covid_df.astype({'total_cases': 'Int64'})
covid_df = covid_df.astype({'total_deaths': 'Int64'})

covid_df = covid_df.astype({'new_cases': 'object'})
covid_df = covid_df.astype({'new_deaths': 'object'})
covid_df = covid_df.astype({'total_cases': 'object'})
covid_df = covid_df.astype({'total_deaths': 'object'})

covid_df.location = covid_df.location.apply(getCountryId)
covid_df = covid_df.dropna(subset=['location'])
covid_df = covid_df.astype({'location': 'int'})

covid_df.index = covid_df.index + 1
covid_df.fillna('\\N', inplace=True)

In [29]:
covid_df

,date,location,new_cases,new_deaths,total_cases,total_deaths
1,2020-02-24,1,1,\N,1,\N
2,2020-02-25,1,0,\N,1,\N
3,2020-02-26,1,0,\N,1,\N
4,2020-02-27,1,0,\N,1,\N
5,2020-02-28,1,0,\N,1,\N
...,...,...,...,...,...,...
84866,2021-05-05,190,30,0,38357,1574
84867,2021-05-06,190,41,1,38398,1575
84868,2021-05-07,190,5,1,38403,1576
84869,2021-05-08,190,11,0,38414,1576


In [30]:
covid_df.to_csv("covid_data.csv") # Remove header for SQL
print("Created 'covid_data.csv'")

Created 'covid_data.csv'
